In [3]:
import numpy as np
import random

#输入矩阵， 语料库中字母的数量是5，使用三维向量表示一个字母
inputVerctors = np.random.randn(5, 3)
#输出矩阵
outputVectors = np.random.randn(5, 3)
#句子
sentence = ['a', 'e', 'd', 'b', 'd', 'c', 'd', 'e', 'e', 'c', 'a']
#中心字母
centerword = 'c'
#上下文字母
context = ['a', 'e', 'd', 'b', 'd', 'd', 'e', 'e', 'c', 'a']
#用于映射字母在输入输出矩阵中的索引
tokens = dict([('a', 0), ('b', 1), ('c', 2), ('d', 3), ('e', 4)])

inputVerctors[tokens['c']]

array([-0.50128876,  0.84007629, -1.48067359])

In [5]:
def softmax(x):
    orig_shape = x.shape
    #根据输入类型是矩阵还是向量分别计算softmax
    if len(x.shape) > 1:
        #矩阵
        tmp = np.max(x, axis=1) #得到每行的最大值，用于缩放每行的元素，避免溢出
        x -= tmp.reshape((x.shape[0], 1)) #使每行减去所在行的最大值（广播运算）
        
        x = np.exp(x) # 第一步，计算所有以e为底的x次幂
        tmp = np.sum(x, axis=1) #将每行求和并保存
        x /= tmp.reshape((x.shape[0], 1)) #所有元素所在行的元素和（广播运算）
    else:
        tmp = np.max(x) #得到最大值
        x -= tmp # 利用最大值缩放数据
        x = np.exp(x) #对所有元素求以e为底的x次幂
        tmp = np.sum(x) #求元素和
        x /= tmp #求softmax
    return x

def sigmoid(x):
    return np.true_divide(1, 1 + np.exp(-x))

# 可以证明：sigmoid函数关于输入x的导数等于`sigmoid(x)(1-sigmoid(x))`
def sigmoid_grad(s):
    return s * (1 - s)

def softmaxCostAndGradient(predicted, target, ouputVectorts):
    v_hat = predicted #中心词向量
    z = np.dot(outputVectors, v_hat) #预测得分
    y_hat = softmax(z) #预测输出y_hat
    
    cost = -np.log(y_hat[target]) #计算代价
    
    z = y_hat.copy()
    z[target] -= 1.0
    grad = np.outer(z, v_hat) # 计算中心词的梯度
    gradPred = np.dot(outputVectors.T, z) #计算输出词向量矩阵的梯度
    
    return cost, gradPred, grad

## 前向传播
def skipgram(currentWord, contextWords, tokens, inputVectors, ouputVectors):
    #初始化变量
    cost = 0
    gradIn = np.zeros(inputVerctors.shape)
    gradOut = np.zeros(outputVectors.shape)
    
    cword_idx = tokens[currentWord] #得到中心词的索引
    v_hat = inputVerctors[cword_idx] #得到中心词的词向量
    
    #循环预测上下文中每个字母
    for j in contextWords:
        u_idx = tokens[j]
        c_cost, c_grad_in, c_grad_out = softmaxCostAndGradient(v_hat, u_idx, outputVectors)
        cost += c_cost #所有代价求和
        gradIn[cword_idx] + c_grad_in # 中心词向量梯度求和
        gradOut += c_grad_out #输出词向量矩阵梯度求和
    
    return cost, gradIn, gradOut

c, gin, gout = skipgram(centerword, context, tokens, inputVerctors, outputVectors)
step = 0.01 #学习速率
print '原始输入矩阵：\n', inputVerctors
print '原始输出矩阵：\n', outputVectors
inputVerctors -= step * gin
outputVectors -= step * gout
print '更新后的输入矩阵：\n', inputVerctors
print '更新后的输出矩阵：\n', outputVectors

原始输入矩阵：
[[ 0.23572163  0.44846602 -0.14915017]
 [ 1.50655112  0.00358611  1.31050247]
 [-0.50128876  0.84007629 -1.48067359]
 [-0.01938465  0.96989823 -0.96163808]
 [ 0.05383894  0.70152992 -1.20296992]]
原始输出矩阵：
[[ 0.16207975 -0.2206265  -0.17491369]
 [-0.28257026 -1.37573974  1.16710697]
 [ 1.02108109 -1.1730129   1.1193676 ]
 [ 2.17922249 -1.76492886 -0.50243915]
 [ 0.38828171  1.79918973  0.05192168]]
更新后的输入矩阵：
[[ 0.23572163  0.44846602 -0.14915017]
 [ 1.50655112  0.00358611  1.31050247]
 [-0.50128876  0.84007629 -1.48067359]
 [-0.01938465  0.96989823 -0.96163808]
 [ 0.05383894  0.70152992 -1.20296992]]
更新后的输出矩阵：
[[ 0.16260495 -0.22150666 -0.17336237]
 [-0.28689816 -1.36848689  1.15432349]
 [ 1.01652165 -1.16537204  1.10590023]
 [ 2.16588731 -1.74258132 -0.54182773]
 [ 0.40997904  1.76282864  0.1160098 ]]
